In [1]:
import azureml.core


# Check core SDK version number.
print('SDK version:', azureml.core.VERSION)

SDK version: 1.7.0


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ANTQYUJ64 to authenticate.
Interactive authentication successfully completed.
hakk-ai
hakk.ai
eastus
b738af92-69d2-4390-a1d8-579dc094b72e


In [15]:
from azureml.core.model import Model

model = Model.register(model_path="Model/final_model.pkl",
                       model_name="finalized_model",
                       workspace=ws)

print('Name:', model.name)
print('Version:', model.version)

Registering model finalized_model
Name: finalized_model
Version: 3


In [16]:
from azureml.core.conda_dependencies import CondaDependencies 


# Usually a good idea to choose specific version numbers
# so training is made on same packages as scoring
myenv = CondaDependencies.create(
    conda_packages=[
        'numpy==1.15.4',
        'scikit-learn==0.19.1', 
        'pandas==0.23.4',        
        'joblib',
        'xgboost'
    ],
    pip_packages=[
        'azureml-defaults>=1.0.45',
        'inference-schema[numpy-support]',
        'inference-schema[pandas-support]'
    ], 
    python_version='3.6.2')

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [20]:
%%writefile score.py
import json
import numpy as np
import joblib
from azureml.core.model import Model
import xgboost a
import pandas as pd

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

def haversine_km(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

def haversine_m(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 3956 * c
    return m

def preprocessing(data):
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
    data['timestamp'] = pd.DatetimeIndex(data.timestamp)

    data['is_weekend'] = np.where(data['day_of_week'].isin([5,6]), 1, 0)
    data['is_weekday'] = np.where(data['day_of_week'].isin([5,6]), 0, 1)

    data['is_wee_hours'] = np.where(data['hour_of_day'].isin([17,18,19,20,21]), 1, 0)

    data['is_rush_hours_morning'] = np.where(data.timestamp.dt.strftime('%H:%M:%S').between('11:30:00', '01:30:00'), 1, 0)
    data['is_rush_hours_evening'] = np.where(data.timestamp.dt.strftime('%H:%M:%S').between('09:00:00', '12:00:00'), 1, 0)

    data['sin_hour_of_day'] = np.sin(2*np.pi*data.hour_of_day/24)
    data['cos_hour_of_day'] = np.cos(2*np.pi*data.hour_of_day/24)
    data['sin_day_of_week'] = np.sin(2*np.pi*data.day_of_week/7)
    data['cos_day_of_week'] = np.cos(2*np.pi*data.day_of_week/7)


    data['haversine_km'] = haversine_km(data['longitude_origin'], data['latitude_origin'], 
                                 data['longitude_destination'], data['latitude_destination'])

    data['haversine_m'] = haversine_m(data['longitude_origin'], data['latitude_origin'], 
                                 data['longitude_destination'], data['latitude_destination'])


    from sklearn.decomposition import PCA
    coords = np.vstack((data[['latitude_origin', 'longitude_origin']].values,
                    data[['latitude_destination', 'longitude_destination']].values))
    
    pca = PCA().fit(coords)

    data['pickup_pca0'] = pca.transform(data[['latitude_origin', 'longitude_origin']])[:, 0]
    data['pickup_pca1'] = pca.transform(data[['latitude_origin', 'longitude_origin']])[:, 1]
    data['dropoff_pca0'] = pca.transform(data[['latitude_destination', 'longitude_destination']])[:, 0]
    data['dropoff_pca1'] = pca.transform(data[['latitude_destination', 'longitude_destination']])[:, 1]

    data = data.drop(['timestamp'], axis=1)
    
    return data

def init():
    global model

    model_path = Model.get_model_path(model_name='finalized_model')
    bst = xgb.Booster({'nthread': 1})  # init model
    model = bst.load_model(model_path)  # load data
    #model = joblib.load(model_path)

    
input_sample = pd.DataFrame(data=[{
    "latitude_origin": -6.141255,
    "longitude_origin": 106.692710,
    "latitude_destination": -6.141150,
    "longitude_destination": 106.693154,
    "timestamp": 1590487113,
    "hour_of_day": 9,
    "day_of_week": 1
}])

# This is an integer type sample. Use the data type that reflects the expected result.
output_sample = np.array([360.00])

# To indicate that we support a variable length of data input,
# set enforce_shape=False
@input_schema('data', PandasParameterType(input_sample, enforce_shape=False))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        #preprocessing
        data = preprocessing(data)
        #result
        result = model.predict(data)    
        return result.tolist()
    
    except Exception as e:
        result = str(e)
        return result

Overwriting score.py


# Deploy locally

In [21]:
from azureml.core import Webservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import LocalWebservice

service_name = 'hakkai-service-test'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

myenv = Environment.from_conda_specification(name="env", file_path="myenv.yml")

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)

deployment_config = LocalWebservice.deploy_configuration(port=8890)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

print(service.port)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry hakkai6fb3fa5e.azurecr.io
Logging into Docker registry hakkai6fb3fa5e.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM hakkai6fb3fa5e.azurecr.io/azureml/azureml_73d03d5fb609b949b5587123a3620e38
 ---> fc9efa5255e2
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 6b8920f79043
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImI3MzhhZjkyLTY5ZDItNDM5MC1hMWQ4LTU3OWRjMDk0YjcyZSIsInJlc291cmNlR3JvdXBOYW1lIjoiaGFray5haSIsImFjY291bnROYW1lIjoiaGFray1haSIsIndvcmtzcGFjZUlkIjoiYTkxYzQ5YjUtN2RlMC00Yzg4LTk3ZDAtOTdiNjVjMzgwODE3In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 76c22b96c220
 ---> 1b8906b5ee67
Step 4/5 : RUN mv '/var/azureml-app/tmpmq2kehhs.py' /var/azureml-app/main.py
 ---> Running in 70cbb8b4e39a
 ---> cc62224f42aa
Step 5/5 : CMD ["runsvdir","/var/runit"]
 ---

ERROR - Error: Container has crashed. Did your init method fail?




Container Logs:
2020-06-19T21:51:17,685909464+00:00 - gunicorn/run 
2020-06-19T21:51:17,688731570+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-06-19T21:51:17,694559081+00:00 - iot-server/run 
2020-06-19T21:51:17,698720889+00:00 - rsys

WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [19]:
print(service.get_logs())

2020-06-19T21:25:49,253973093+00:00 - rsyslog/run 
2020-06-19T21:25:49,257155899+00:00 - gunicorn/run 
2020-06-19T21:25:49,257777400+00:00 - iot-server/run 
2020-06-19T21:25:49,258911602+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [ ]:
print(service.state)

# Deploy on cloud

In [13]:
from azureml.core import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'finalized-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass


myenv = Environment.from_conda_specification(name="env", file_path="myenv.yml")

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)
aci_config = AciWebservice.deploy_configuration(cpu_cores=3, memory_gb=15, enable_app_insights=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

Running..............................................................................................................................................................................................................................................................................................................

ERROR - Received bad response from Resource Provider:
Response Code: 500
Headers: {'Date': 'Fri, 19 Jun 2020 21:07:33 GMT', 'Content-Type': 'text/html', 'Content-Length': '170', 'Connection': 'close', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'<html>\r\n<head><title>500 Internal Server Error</title></head>\r\n<body>\r\n<center><h1>500 Internal Server Error</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'

ERROR - Received bad response from Resource Provider:
Response Code: 500
Headers: {'Date': 'Fri, 19 Jun 2020 21:07:33 GMT', 'Content-Type': 'text/html', 'Content-Length': '170', 'Connection': 'close', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'<html>\r\n<head><title>500 Internal Server Error</title></head>\r\n<body>\r\n<center><h1>500 Internal Server Error</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'



WebserviceException: WebserviceException:
	Message: Received bad response from Resource Provider:
Response Code: 500
Headers: {'Date': 'Fri, 19 Jun 2020 21:07:33 GMT', 'Content-Type': 'text/html', 'Content-Length': '170', 'Connection': 'close', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'<html>\r\n<head><title>500 Internal Server Error</title></head>\r\n<body>\r\n<center><h1>500 Internal Server Error</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from Resource Provider:\nResponse Code: 500\nHeaders: {'Date': 'Fri, 19 Jun 2020 21:07:33 GMT', 'Content-Type': 'text/html', 'Content-Length': '170', 'Connection': 'close', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}\nContent: b'<html>\\r\\n<head><title>500 Internal Server Error</title></head>\\r\\n<body>\\r\\n<center><h1>500 Internal Server Error</h

In [14]:
print(service.get_logs())

/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-06-19T21:09:27,516022071+00:00 - iot-server/run 
2020-06-19T21:09:27,517363377+00:00 - gunicorn/run 
2020-06-19T21:09:27,518810284+00:00 - rsyslog/run 
2020-06-19T21:09:27,523375505+00:00 - nginx/run 
EdgeHubC

In [ ]:
print(service.swagger_uri)

In [ ]:
print(service.scoring_uri)